In [ ]:
# Get pipeline run id
pipeline_run_id = ''

In [ ]:
import os
import datetime

# Get keyvault linked service name
keyvaultlsname = 'Ls_KeyVault_01'

token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary

# Primary storage info 
storage_account_url = token_library.getSecretWithLS(keyvaultlsname, "datalakeurl").replace("https://", '')
container_name = 'saveddata' # fill in your container name 
relative_path = '' # fill in your relative folder path 
file_name = f"{pipeline_run_id}.parquet"
adls_path = 'abfss://%s@%s/%s' % (container_name, storage_account_url, relative_path) 
print('Primary storage account path: ' + adls_path) 
loaded_on = datetime.datetime.now()
base_path = os.path.join(adls_path, file_name)
print('Base path is: ' + base_path)
df = spark.read.parquet(base_path)

df.printSchema()
df.show()

In [ ]:
count = df.count()
data = [[loaded_on, file_name, pipeline_run_id, df.count()]]
 
# specify column names
columns = ['date', 'filename', 'pipeline_run_id', 'count']
 
# creating a dataframe from the lists of data
dataframe = spark.createDataFrame(data, columns)

dataframe.show()

In [ ]:
dataframe.write.mode('append').parquet(os.path.join(adls_path, 'interim.count'))